# Totale

##### setup

In [264]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf
import openpyxl

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2
    complePath = '/home/alinux/unibg/git_repo/SCGProject/Datasets/'

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")



dfCambio["TassoCambioMedio"] = dfCambio["TassoCambioMedio"].str.replace(',','.')


Username:  davidguzman


# Ricavi

In [265]:
dfVendite.rename(columns = {'budget/cons':'budget'}, inplace = True)
venditeConsuntivo = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'CONSUNTIVO' or budget = 'Consuntivo'")
venditeBudget = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'BUDGET' or budget = 'Budget'")

In [266]:
print(venditeConsuntivo.shape, venditeBudget.shape)

(1767, 7) (1209, 7)


In [267]:
# join with dfCambio and dfClienti
venditeConsuntivo = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
FROM venditeConsuntivo as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
where k.Anno = 'Consuntivo' or k.Anno = 'CONSUNTIVO'
''')

venditeBudget = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
FROM venditeBudget as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
where k.Anno = 'Budget' or k.Anno = 'BUDGET'
''')

In [268]:
print(venditeConsuntivo.shape, venditeBudget.shape)

(1767, 11) (1209, 11)


In [269]:
sumVenditeConsuntivo = sqldf('select sum(TotaleEuro) as sumVenditaTotale from venditeConsuntivo')
sumVenditeBudget = sqldf('select sum(TotaleEuro) as sumVenditaTotale from venditeBudget')
print(sumVenditeConsuntivo)
print(sumVenditeBudget)
print(sumVenditeConsuntivo-sumVenditeBudget) #questo è lo scostamento totale dei ricavi

   sumVenditaTotale
0     507842.851413
   sumVenditaTotale
0     395335.892443
   sumVenditaTotale
0      112506.95897


# Costi

Materie prime

In [270]:
dfConsumi.rename(columns = {'Budget/cons':'budget'}, inplace = True)
consumiConsuntivo = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'CONSUNTIVO'")
consumiBudget = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'BUDGET'")
sumConsumiConsuntivo = sqldf('select sum(ImportoCostoTOTALE) from consumiConsuntivo')
sumConsumiBudget = sqldf('select sum(ImportoCostoTOTALE) from consumiBudget')
print(sumConsumiConsuntivo)
print(sumConsumiBudget)
print(sumConsumiConsuntivo-sumConsumiBudget) #questo è lo scostamento totale della materia prima

   sum(ImportoCostoTOTALE)
0                 289639.5
   sum(ImportoCostoTOTALE)
0                192906.47
   sum(ImportoCostoTOTALE)
0                 96733.03


Lavorazioni interne

In [271]:
impiegoConsuntivo = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'CONSUNTIVO' or budgetConsuntivo = 'Consuntivo'")
print(impiegoConsuntivo.shape)
#join con i costi orari
impiegoConsuntivo  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoConsuntivo as i join dfCostoOrarioConsuntivo as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
print(impiegoConsuntivo.shape)
# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoConsuntivoOld = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoConsuntivo1 = impiegoConsuntivoOld.copy()
# ASSUNZIONE: la quantity di output maggiore tiene conto di tutte quelle inferiori.


(3791, 9)
(3723, 9)


In [272]:
impiegoConsuntivoNew = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, max(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoConsuntivoNew

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,2.0
1,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.0000,50,2.0
2,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0000,80,2.0
3,ART0000041,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,4.0
4,ART0000041,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS1,0.0001,80,4.0
...,...,...,...,...,...,...,...,...,...
3012,ART0005136,CONSUNTIVO,ODP19-06532,Fresatura,A30,RIS14,1.2500,50,3.0
3013,ART0005136,CONSUNTIVO,ODP19-06533,Controllo qualità,CQ,RIS16,0.0000,15,4.0
3014,ART0005136,CONSUNTIVO,ODP19-06533,Tornitura,A20,RIS13,0.5000,80,4.0
3015,ART0005136,CONSUNTIVO,ODP19-06688,Controllo qualità,CQ,RIS16,0.0000,15,9.0


In [273]:
print(sqldf('select sum(QuantitydiOutput) from impiegoConsuntivoOld'))
print(sqldf('select sum(QuantitydiOutput) from impiegoConsuntivoNew'))

   sum(QuantitydiOutput)
0                25950.0
   sum(QuantitydiOutput)
0                25327.0


In [274]:
sqldf('select NrArticolo, NrOrdineProduzione, count(*) as c from impiegoConsuntivoNew group by NrArticolo, NrOrdineProduzione order by c')

,NrArticolo,NrOrdineProduzione,c
0,ART0000116,ODP18-03472,1
1,ART0000116,ODP18-05889,1
2,ART0000119,ODP19-04430,1
3,ART0000121,ODP18-03469,1
4,ART0000128,ODP18-03529,1
...,...,...,...
1236,ART0005136,ODP19-01954,5
1237,ART0000043,ODP19-05998,6
1238,ART0001731,ODP18-04233,6
1239,ART0002360,ODP19-06224,6


In [275]:
impiegoBudget = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget'")
#join con i costi orari
print(impiegoBudget.shape)
impiegoBudget = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoBudget as i join dfCostoOrarioBudget as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
print(impiegoBudget.shape)
# righe a consuntivo sono di più che a budget e la somma dei 2 non corrisponde al numero di righe della tabella impiego risorse

# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoBudgetOld = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoBudget1 = impiegoBudgetOld.copy()

impiegoBudgetNew = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, max(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')


(2473, 9)
(2450, 9)


In [276]:
print(sqldf('select sum(QuantitydiOutput) from impiegoBudgetOld'))
print(sqldf('select sum(QuantitydiOutput) from impiegoBudgetNew'))

   sum(QuantitydiOutput)
0                15768.5
   sum(QuantitydiOutput)
0               15580.25


In [277]:
impiegoBudgetNew = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudgetNew
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')

In [278]:
impiegoBudgetNew

sqldf('''select NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa, count(*)
from impiegoBudgetNew 
group by NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa
order by count(*) desc''')

sqldf('''select NrArticolo, Descrizione, NrAreaProduzione, Risorsa, count(*)
from impiegoBudgetNew 
group by NrArticolo, Descrizione, NrAreaProduzione, Risorsa
order by count(*) desc''')

,NrArticolo,Descrizione,NrAreaProduzione,Risorsa,count(*)
0,ART0005136,Controllo qualità,CQ,RIS16,19
1,ART0005136,Tornitura,A20,RIS2,13
2,ART0000795,Controllo qualità,CQ,RIS16,11
3,ART0000762,Controllo qualità,CQ,RIS16,9
4,ART0000782,Controllo qualità,CQ,RIS16,8
...,...,...,...,...,...
1395,ART0005136,Brocciatura / stozzatrice,A33,RIS2,1
1396,ART0005136,Foratura/Maschiatura,A32,RIS9,1
1397,ART0005136,Fresatura,A30,RIS1,1
1398,ART0005136,Fresatura,A30,RIS13,1


# Scostamento totale

In [279]:
sumImpiegoConsuntivo = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoConsuntivoNew WHERE TempoRisorsa>=0')
sumImpiegoBudget = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoBudgetNew WHERE TempoRisorsa>=0')
print(sumImpiegoConsuntivo)
print(sumImpiegoBudget)
print(sumImpiegoConsuntivo-sumImpiegoBudget)

   sum(TempoRisorsa*CostoOrario)
0                    359238.2962
   sum(TempoRisorsa*CostoOrario)
0                   194521.50346
   sum(TempoRisorsa*CostoOrario)
0                   164716.79274


In [280]:
print(sumVenditeConsuntivo.values[0][0])
print(sumImpiegoConsuntivo.values[0][0]+sumConsumiConsuntivo.values[0][0])
print(sumVenditeConsuntivo.values[0][0]-sumImpiegoConsuntivo.values[0][0]-sumConsumiConsuntivo.values[0][0])
print(sumVenditeBudget.values[0][0])
print(sumImpiegoBudget.values[0][0]+sumConsumiBudget.values[0][0])
print(sumVenditeBudget.values[0][0]-sumImpiegoBudget.values[0][0]-sumConsumiBudget.values[0][0])

507842.8514127266
648877.7962
-141034.94478727342
395335.8924431994
387427.97346000024
7907.918983199197


Senza vincoli su tempi negativi

**non cambia perché per ogni tempo negativo esiste un tempo positivo uguale in modulo che, al sommare i tempi delle stesse attività, si annullano a vicenda**

In [281]:
sumImpiegoConsuntivo = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoConsuntivoNew')
sumImpiegoBudget = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoBudgetNew')
print(sumImpiegoConsuntivo)
print(sumImpiegoBudget)
print(sumImpiegoConsuntivo-sumImpiegoBudget)

   sum(TempoRisorsa*CostoOrario)
0                    359215.7962
   sum(TempoRisorsa*CostoOrario)
0                   194521.50346
   sum(TempoRisorsa*CostoOrario)
0                   164694.29274


In [282]:
print(sumVenditeConsuntivo.values[0][0])
print(sumImpiegoConsuntivo.values[0][0]+sumConsumiConsuntivo.values[0][0])
print(sumVenditeConsuntivo.values[0][0]-sumImpiegoConsuntivo.values[0][0]-sumConsumiConsuntivo.values[0][0])
print(sumVenditeBudget.values[0][0])
print(sumImpiegoBudget.values[0][0]+sumConsumiBudget.values[0][0])
print(sumVenditeBudget.values[0][0]-sumImpiegoBudget.values[0][0]-sumConsumiBudget.values[0][0])


507842.8514127266
648855.2962
-141012.44478727342
395335.8924431994
387427.97346000024
7907.918983199197


# Socostamento di volume, impiego e prezzo per le Lavorazioni

#### consuntivo

In [283]:
# Creazione colonna ImpiegoUnitario
impiegoConsuntivoNew = sqldf(''' select NrArticolo, budgetConsuntivo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa, TempoRisorsa, TempoRisorsa/QuantitydiOutput as ImpiegoUnitario, CostoOrario ,QuantitydiOutput
from impiegoConsuntivoNew''')
impiegoConsuntivoNew

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput
0,ART0000018,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,0.000000,15,2.0
1,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.0000,0.500000,50,2.0
2,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0000,0.500000,80,2.0
3,ART0000041,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,0.000000,15,4.0
4,ART0000041,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS1,0.0001,0.000025,80,4.0
...,...,...,...,...,...,...,...,...,...,...
3012,ART0005136,CONSUNTIVO,ODP19-06532,Fresatura,A30,RIS14,1.2500,0.416667,50,3.0
3013,ART0005136,CONSUNTIVO,ODP19-06533,Controllo qualità,CQ,RIS16,0.0000,0.000000,15,4.0
3014,ART0005136,CONSUNTIVO,ODP19-06533,Tornitura,A20,RIS13,0.5000,0.125000,80,4.0
3015,ART0005136,CONSUNTIVO,ODP19-06688,Controllo qualità,CQ,RIS16,0.0000,0.000000,15,9.0


Avg out the null values

In [284]:
impiegoConsuntivoNew = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, sum(TempoRisorsa) as TempoRisorsa, avg(CostoOrario) as CostoOrario, max(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivoNew
group by NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione
''')
impiegoConsuntivo = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, TempoRisorsa, (TempoRisorsa/QuantitydiOutput) as ImpiegoUnitario, CostoOrario,QuantitydiOutput
from impiegoConsuntivoNew
order by NrArticolo
''')

In [285]:
sqldf('select sum(TempoRisorsa*CostoOrario) from impiegoConsuntivo') # si mantiene quasi uguale!!

,sum(TempoRisorsa*CostoOrario)
0,359425.545873


In [286]:
sqldf('select sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput) from impiegoConsuntivo') 

,sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput)
0,358735.055873


la piccola differenza è dovuta alla restante parte di righe con output nulle e senza altre righe con output non nullo da sommare

In [287]:
sqldf('select * from impiegoConsuntivo where ImpiegoUnitario is null') 

,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput
0,ART0000151,CONSUNTIVO,Fresatura,A30,3.0000,None,50.0,0.0
1,ART0001789,CONSUNTIVO,Fresatura,A30,4.4990,None,50.0,0.0
2,ART0001872,CONSUNTIVO,Fresatura,A30,0.0001,None,80.0,0.0
3,ART0001873,CONSUNTIVO,Fresatura,A30,0.0001,None,80.0,0.0
4,ART0001939,CONSUNTIVO,Fresatura,A30,0.0001,None,80.0,0.0
5,ART0001942,CONSUNTIVO,Fresatura,A30,0.0001,None,80.0,0.0
6,ART0001945,CONSUNTIVO,Fresatura,A30,0.0001,None,80.0,0.0
7,ART0002152,CONSUNTIVO,Fresatura,A30,4.0000,None,50.0,0.0
8,ART0004146,CONSUNTIVO,Montaggio,A40,1.5000,None,50.0,0.0
9,ART0004357,CONSUNTIVO,Fresatura,A30,0.0100,None,50.0,0.0


#### ripetiamo per budget

In [288]:
impiegoBudgetNew = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, sum(TempoRisorsa) as TempoRisorsa, avg(CostoOrario) as CostoOrario, max(QuantitydiOutput) as QuantitydiOutput
from impiegoBudgetNew
group by NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione
''')
impiegoBudget = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, TempoRisorsa, (TempoRisorsa/QuantitydiOutput) as ImpiegoUnitario, CostoOrario,QuantitydiOutput
from impiegoBudgetNew
order by NrArticolo
''')

In [289]:
sqldf('select sum(TempoRisorsa*CostoOrario) from impiegoBudget') # si mantiene quasi uguale!!

,sum(TempoRisorsa*CostoOrario)
0,194739.149842


In [290]:
sqldf('select sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput) from impiegoBudget') 

,sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput)
0,192493.776242


In [291]:
sqldf('select * from impiegoBudget where ImpiegoUnitario is null') 

,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput
0,ART0000129,BUDGET,Fresatura,A30,0.0090,None,68.0,0.0
1,ART0000157,BUDGET,Tornitura,A20,15.0000,None,68.0,0.0
2,ART0000645,BUDGET,Fresatura,A30,1.0019,None,68.0,0.0
3,ART0000883,BUDGET,Fresatura,A30,1.0000,None,42.5,0.0
4,ART0001866,BUDGET,Rettifica,A34,10.0000,None,42.5,0.0
5,ART0001877,BUDGET,Fresatura,A30,0.0001,None,68.0,0.0
6,ART0001885,BUDGET,Fresatura,A30,0.0001,None,68.0,0.0
7,ART0001917,BUDGET,Fresatura,A30,0.0090,None,68.0,0.0
8,ART0003780,BUDGET,Fresatura,A30,1.5000,None,51.0,0.0
9,ART0004084,BUDGET,Montaggio,A40,3.1500,None,42.5,0.0


#### Merge/Join

In [292]:
impiegoConsuntivo.drop(['budgetConsuntivo'], axis=1, inplace=True)
impiegoBudget.drop(['budgetConsuntivo'], axis=1, inplace=True)
x = pd.merge(impiegoConsuntivo, impiegoBudget, on=['NrArticolo', 'Descrizione', 'NrAreaProduzione'], how='outer')
x

,NrArticolo,Descrizione,NrAreaProduzione,TempoRisorsa_x,ImpiegoUnitario_x,CostoOrario_x,QuantitydiOutput_x,TempoRisorsa_y,ImpiegoUnitario_y,CostoOrario_y,QuantitydiOutput_y
0,ART0000018,Controllo qualità,CQ,0.0000,0.000000,15.0,2.0,0.00,0.000000,12.75,2.0
1,ART0000018,Fresatura,A30,1.0000,0.500000,50.0,2.0,0.75,0.375000,51.00,2.0
2,ART0000018,Tornitura,A20,1.0000,0.500000,80.0,2.0,1.00,0.500000,68.00,2.0
3,ART0000041,Controllo qualità,CQ,0.0000,0.000000,15.0,4.0,0.00,0.000000,12.75,4.0
4,ART0000041,Fresatura,A30,0.0001,0.000025,80.0,4.0,1.25,0.312500,51.00,4.0
...,...,...,...,...,...,...,...,...,...,...,...
1345,ART0004904,Tornitura,A20,NaN,NaN,NaN,NaN,1.75,0.437500,68.00,4.0
1346,ART0004944,Tornitura,A20,NaN,NaN,NaN,NaN,1.75,0.218750,68.00,8.0
1347,ART0005091,Fresatura,A30,NaN,NaN,NaN,NaN,1.00,0.250000,42.50,4.0
1348,ART0005136,Foratura/Maschiatura,A32,NaN,NaN,NaN,NaN,3.00,NaN,42.50,0.0


In [293]:
c = sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*QuantitydiOutput_x) from x').iloc[0][0]
b = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_y) from x').iloc[0][0]
print(c)
print(b)
print(c-b)
# non altera i totali!!

358735.05587289936
192493.7762416667
166241.27963123267


In [294]:
#Budget Standard
BudgetStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_y) from x').iloc[0][0]
# Impiego Standard (volume a consuntivo)
ImpiegoStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_x) from x').iloc[0][0]
# Impiego Effettivo (volume e impiego a consuntivo)
ImpiegoEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_y*QuantitydiOutput_x) from x').iloc[0][0]
# Budget Effettivo (tutto a consuntivo)
BudgetEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*QuantitydiOutput_x) from x').iloc[0][0]
print(BudgetStandardLav)
print(ImpiegoStandardLav)
print(ImpiegoEffettivoLav)
print(BudgetEffettivoLav)

192493.7762416667
250080.51605938777
281289.91565333324
358735.05587289936


In [297]:
sqldf('select ImpiegoUnitario_y,CostoOrario_y,QuantitydiOutput_x from x order by QuantitydiOutput_x desc')

,ImpiegoUnitario_y,CostoOrario_y,QuantitydiOutput_x
0,0.000000,12.750000,116.0
1,0.020455,68.000000,116.0
2,0.000000,12.750000,115.0
3,1.047170,63.750000,115.0
4,0.129167,53.833333,110.0
...,...,...,...
1345,0.437500,68.000000,NaN
1346,0.218750,68.000000,NaN
1347,0.250000,42.500000,NaN
1348,NaN,42.500000,NaN


In [295]:
#Scostamenti
ScostamentoVolumeLav = ImpiegoStandardLav-b
ScostamentoImpiegoLav = ImpiegoEffettivoLav-ImpiegoStandardLav
ScostamentoPrezzoLav = BudgetEffettivoLav-ImpiegoEffettivoLav
print(ScostamentoVolumeLav)
print(ScostamentoImpiegoLav)
print(ScostamentoPrezzoLav)
print(ScostamentoVolumeLav+ScostamentoImpiegoLav+ScostamentoPrezzoLav)

57586.73981772107
31209.399593945476
77445.14021956612
166241.27963123267


# Socostamento di volume, impiego e prezzo per le materie prime

#### Group by CodiceMP

se non facciamo qualche group by a priori si sdoppiano le righe, proviamo prima con CodiceMP

In [172]:
consumiConsuntivoUnitByMP = sqldf('''select CodiceMP, NrArticolo, NrDocumento as NrOrdineProduzione, sum(QuantityMPImpiegata) as QuantityMPImpiegata, sum(ImportoCostoTOTALE) as ImportoCostoTOTALE
from consumiConsuntivo     
group by CodiceMP, NrArticolo, NrOrdineProduzione''')
consumiBudgetUnitByMP = sqldf('''select CodiceMP, NrArticolo,NrDocumento as NrOrdineProduzione, sum(QuantityMPImpiegata) as QuantityMPImpiegata, sum(ImportoCostoTOTALE) as ImportoCostoTOTALE
from consumiBudget     
group by CodiceMP, NrArticolo,NrOrdineProduzione''')
consumiConsuntivoUnitByMP

,CodiceMP,NrArticolo,NrOrdineProduzione,QuantityMPImpiegata,ImportoCostoTOTALE
0,ART0000771,ART0005136,ODP19-01043,1.0,230.95
1,ART0000809,ART0005136,ODP19-01043,2.0,1122.70
2,ART0001066,ART0003099,ODP18-08457,1064.0,1.99
3,ART0001068,ART0004671,ODP19-04541,190.0,6.84
4,ART0001073,ART0002240,ODP18-06051,184.0,15.95
...,...,...,...,...,...
2766,ART0007130,ART0001228,ODP19-06232,15.0,22.50
2767,ART0007133,ART0005136,ODP19-06532,3.0,0.00
2768,ART0007141,ART0002191,ODP19-06418,2.0,13.00
2769,ART0007148,ART0002619,ODP19-05754,2.0,60.00


##### ritrovare volumi di produzione

In [173]:
temp = sqldf('''select a.CodiceMP,a.NrArticolo,a.NrOrdineProduzione,a.QuantityMPImpiegata,a.ImportoCostoTOTALE,
 (select max(QuantitydiOutput) from impiegoConsuntivoNew  where a.NrArticolo = NrArticolo and a.NrOrdineProduzione = NrOrdineProduzione) as QuantitydiOutput
 from consumiConsuntivoUnitByMP as a''')
temp = sqldf('''select distinct CodiceMP,NrArticolo,sum(QuantityMPImpiegata) as QuantityMPImpiegata,sum(ImportoCostoTOTALE) as ImportoCostoTOTALE ,QuantitydiOutput 
from temp group by CodiceMP,NrArticolo''')
consumiConsuntivoVolProd = sqldf('''select distinct CodiceMP,NrArticolo,QuantityMPImpiegata,QuantitydiOutput,(ImportoCostoTOTALE/QuantityMPImpiegata)/QuantitydiOutput as CostoUnitario, ImportoCostoTOTALE
from temp group by CodiceMP,NrArticolo''')

temp = sqldf('''select a.CodiceMP,a.NrArticolo,a.NrOrdineProduzione,a.QuantityMPImpiegata,a.ImportoCostoTOTALE,
 (select max(QuantitydiOutput) from impiegoBudgetNew  where a.NrArticolo = NrArticolo and a.NrOrdineProduzione = NrOrdineProduzione) as QuantitydiOutput
 from consumiBudgetUnitByMP as a''')
temp = sqldf('''select distinct CodiceMP,NrArticolo,sum(QuantityMPImpiegata) as QuantityMPImpiegata,sum(ImportoCostoTOTALE) as ImportoCostoTOTALE ,QuantitydiOutput 
from temp group by CodiceMP,NrArticolo''')
consumiBudgetVolProd = sqldf('''select distinct CodiceMP,NrArticolo,QuantityMPImpiegata,QuantitydiOutput,(ImportoCostoTOTALE/QuantityMPImpiegata)/QuantitydiOutput as CostoUnitario, ImportoCostoTOTALE
from temp group by CodiceMP,NrArticolo''')

consumiConsuntivoVolProd

,CodiceMP,NrArticolo,QuantityMPImpiegata,QuantitydiOutput,CostoUnitario,ImportoCostoTOTALE
0,ART0000771,ART0005136,1.0,2.0,115.475000,230.95
1,ART0000809,ART0005136,2.0,2.0,280.675000,1122.70
2,ART0001066,ART0003099,1064.0,19.0,0.000098,1.99
3,ART0001068,ART0004671,190.0,5.0,0.007200,6.84
4,ART0001073,ART0002240,184.0,4.0,0.021671,15.95
...,...,...,...,...,...,...
1074,ART0007130,ART0001228,15.0,15.0,0.100000,22.50
1075,ART0007133,ART0005136,3.0,3.0,0.000000,0.00
1076,ART0007141,ART0002191,2.0,2.0,3.250000,13.00
1077,ART0007148,ART0002619,2.0,2.0,15.000000,60.00


In [174]:
print(consumiConsuntivoVolProd.shape, consumiConsuntivoUnitByMP.shape)
print(consumiBudgetVolProd.shape, consumiBudgetVolProd.shape)

(1079, 6) (2771, 5)
(937, 6) (937, 6)


In [175]:
print(sqldf('select * from consumiConsuntivoVolProd where CostoUnitario is null').shape)
print(sqldf('select * from consumiBudgetVolProd where CostoUnitario is null').shape)
# TODO rimuovere la maggior parte di quanitity nulle possibile dalle lavorazioni continua a essere importante!

(10, 6)
(53, 6)


In [176]:
print(sqldf('select sum(ImportoCostoTOTALE) from consumiConsuntivoUnitByMP'))
print(sqldf('select sum(QuantityMPImpiegata*CostoUnitario*QuantitydiOutput) from consumiConsuntivoVolProd'))
print(sqldf('select sum(ImportoCostoTOTALE) from consumiBudgetUnitByMP'))
print(sqldf('select sum(QuantityMPImpiegata*CostoUnitario*QuantitydiOutput) from consumiBudgetVolProd'))
# non si perdono dei decimali 

   sum(ImportoCostoTOTALE)
0                 289639.5
   sum(QuantityMPImpiegata*CostoUnitario*QuantitydiOutput)
0                                          289033.53      
   sum(ImportoCostoTOTALE)
0                192906.47
   sum(QuantityMPImpiegata*CostoUnitario*QuantitydiOutput)
0                                          191455.42      


##### Scostamenti

In [177]:
#consumiConsuntivoVolProd.drop(columns='NrOrdineProduzione',inplace=True)
#consumiBudgetVolProd.drop(columns='NrOrdineProduzione',inplace=True)

In [178]:
ConsumiUnit = pd.merge(consumiConsuntivoVolProd,consumiBudgetVolProd, on = ['CodiceMP','NrArticolo'], how = 'outer')
ConsumiUnit

,CodiceMP,NrArticolo,QuantityMPImpiegata_x,QuantitydiOutput_x,CostoUnitario_x,ImportoCostoTOTALE_x,QuantityMPImpiegata_y,QuantitydiOutput_y,CostoUnitario_y,ImportoCostoTOTALE_y
0,ART0000771,ART0005136,1.0,2.0,115.475000,230.95,NaN,NaN,NaN,NaN
1,ART0000809,ART0005136,2.0,2.0,280.675000,1122.70,NaN,NaN,NaN,NaN
2,ART0001066,ART0003099,1064.0,19.0,0.000098,1.99,NaN,NaN,NaN,NaN
3,ART0001068,ART0004671,190.0,5.0,0.007200,6.84,NaN,NaN,NaN,NaN
4,ART0001073,ART0002240,184.0,4.0,0.021671,15.95,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1310,ART0005913,ART0002191,NaN,NaN,NaN,NaN,40.0,2.0,0.169625,13.57
1311,ART0005917,ART0001610,NaN,NaN,NaN,NaN,972.0,4.0,0.000149,0.58
1312,ART0005930,ART0000229,NaN,NaN,NaN,NaN,1560.0,12.0,0.000422,7.90
1313,ART0005935,ART0004675,NaN,NaN,NaN,NaN,225.0,5.0,0.002862,3.22


In [179]:
BudgetEffettivoMP = sqldf('select sum(QuantitydiOutput_x*QuantityMPImpiegata_x*CostoUnitario_x) from ConsumiUnit').iloc[0][0]
ImpiegoEffettivoMP = sqldf('select sum(QuantitydiOutput_x*QuantityMPImpiegata_x*CostoUnitario_y) from ConsumiUnit').iloc[0][0]
ImpiegoStandardMP = sqldf('select sum(QuantitydiOutput_x*QuantityMPImpiegata_y*CostoUnitario_y) from ConsumiUnit').iloc[0][0]
BudgetStandardMP = sqldf('select sum(QuantitydiOutput_y*QuantityMPImpiegata_y*CostoUnitario_y) from ConsumiUnit').iloc[0][0]
ScostamentoVolumeMP =  ImpiegoStandardMP - BudgetStandardMP
ScostamentoImpiegoMP = ImpiegoEffettivoMP-ImpiegoStandardMP
ScostamentoPrezzoMP = BudgetEffettivoMP-ImpiegoEffettivoMP 

In [180]:
print(BudgetEffettivoMP)
print(ImpiegoEffettivoMP)
print(ImpiegoStandardMP)
print(BudgetStandardMP)
print(ScostamentoVolumeMP)
print(ScostamentoImpiegoMP)
print(ScostamentoPrezzoMP)
print(ScostamentoVolumeMP+ScostamentoImpiegoMP+ScostamentoPrezzoMP) 

289033.53000000014
373323.63085213234
220739.127731121
191455.41999999998
29283.707731121016
152584.50312101134
-84290.1008521322
97578.11000000016


# Socostamento di volume, mix e prezzo per i ricavi

 group by NrArticolo dei ricavi

In [58]:
venditeConsuntivoArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity, sum(TotaleEuro) as TotaleEuro
        from venditeConsuntivo
        group by NrArticolo''')
venditeConsuntivoArt

,NrArticolo,Quantity,TotaleEuro
0,ART0000018,2,91.720170
1,ART0000041,4,400.233470
2,ART0000042,4,600.350204
3,ART0000043,2,250.145918
4,ART0000115,3,1095.000000
...,...,...,...
408,ART0005109,9,210.122572
409,ART0005110,9,435.253898
410,ART0005111,7,204.285833
411,ART0005112,7,262.653214


In [59]:
venditeBudgetArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity, sum(TotaleEuro) as TotaleEuro
        from venditeBudget
        group by NrArticolo''')
venditeBudgetArt

,NrArticolo,Quantity,TotaleEuro
0,ART0000018,2,85.380894
1,ART0000041,4,360.497107
2,ART0000042,4,569.205958
3,ART0000043,2,218.195617
4,ART0000115,12,1965.000000
...,...,...,...
408,ART0005109,4,106.251779
409,ART0005110,2,94.867660
410,ART0005111,2,66.407362
411,ART0005112,2,85.380894


##### calcolo del delta dei volumi 

In [60]:

vendite = pd.merge(venditeConsuntivoArt,venditeBudgetArt, on = ['NrArticolo'], how = 'outer')
vendite

,NrArticolo,Quantity_x,TotaleEuro_x,Quantity_y,TotaleEuro_y
0,ART0000018,2,91.720170,2,85.380894
1,ART0000041,4,400.233470,4,360.497107
2,ART0000042,4,600.350204,4,569.205958
3,ART0000043,2,250.145918,2,218.195617
4,ART0000115,3,1095.000000,12,1965.000000
...,...,...,...,...,...
408,ART0005109,9,210.122572,4,106.251779
409,ART0005110,9,435.253898,2,94.867660
410,ART0005111,7,204.285833,2,66.407362
411,ART0005112,7,262.653214,2,85.380894


In [61]:
sqldf('''select sum(TotaleEuro_x) from vendite''')

,sum(TotaleEuro_x)
0,507842.851413


In [62]:
sqldf('''select sum(TotaleEuro_y) from vendite''')

,sum(TotaleEuro_y)
0,395335.892443


In [63]:
sqldf('''select NrArticolo, Quantity_x-Quantity_y as DiffQuantity
        from vendite
        order by DiffQuantity desc
        ''')

,NrArticolo,DiffQuantity
0,ART0005136,1046
1,ART0000748,291
2,ART0001575,200
3,ART0004397,114
4,ART0000229,113
...,...,...
408,ART0001628,-96
409,ART0004363,-114
410,ART0001154,-176
411,ART0003610,-176


In [64]:
sqldf('''select sum(Quantity_x-Quantity_y) as DiffQuantity
        from vendite
        ''')

,DiffQuantity
0,2105


##### creazione colonne mix, volume, prezzo unitario

In [65]:
venditeBudgetS = sqldf('''select NrArticolo, (Quantity*100000000000/7224) as Mix, 7224 as VolumeTotale, TotaleEuro/Quantity as PrezzoUnitario
        from venditeBudgetArt''')
venditeBudgetS

,NrArticolo,Mix,VolumeTotale,PrezzoUnitario
0,ART0000018,27685492,7224,42.690447
1,ART0000041,55370985,7224,90.124277
2,ART0000042,55370985,7224,142.301489
3,ART0000043,27685492,7224,109.097809
4,ART0000115,166112956,7224,163.750000
...,...,...,...,...
408,ART0005109,55370985,7224,26.562945
409,ART0005110,27685492,7224,47.433830
410,ART0005111,27685492,7224,33.203681
411,ART0005112,27685492,7224,42.690447


In [66]:
sqldf('select sum(Mix*VolumeTotale*PrezzoUnitario)/100000000000 from venditeBudgetS')

,sum(Mix*VolumeTotale*PrezzoUnitario)/100000000000
0,395335.890784


In [67]:
sqldf('select sum(Quantity) from venditeConsuntivo')

,sum(Quantity)
0,9329


In [68]:
venditeConsuntivoS = sqldf('''select NrArticolo, (Quantity*100000000000/9329) as Mix, 9329 as VolumeTotale, TotaleEuro/Quantity as PrezzoUnitario
        from venditeConsuntivoArt''')
venditeConsuntivoS

,NrArticolo,Mix,VolumeTotale,PrezzoUnitario
0,ART0000018,21438525,9329,45.860085
1,ART0000041,42877050,9329,100.058367
2,ART0000042,42877050,9329,150.087551
3,ART0000043,21438525,9329,125.072959
4,ART0000115,32157787,9329,365.000000
...,...,...,...,...
408,ART0005109,96473362,9329,23.346952
409,ART0005110,96473362,9329,48.361544
410,ART0005111,75034837,9329,29.183690
411,ART0005112,75034837,9329,37.521888


In [69]:
sqldf('select sum(Mix*VolumeTotale*PrezzoUnitario)/100000000000 from venditeConsuntivoS')

,sum(Mix*VolumeTotale*PrezzoUnitario)/100000000000
0,507842.849547


### Calcolo socostamenti di volume, mix e prezzo

In [70]:
venditeS = pd.merge(venditeConsuntivoS, venditeBudgetS, on = ['NrArticolo'], how = 'outer')
venditeS

,NrArticolo,Mix_x,VolumeTotale_x,PrezzoUnitario_x,Mix_y,VolumeTotale_y,PrezzoUnitario_y
0,ART0000018,21438525,9329,45.860085,27685492,7224,42.690447
1,ART0000041,42877050,9329,100.058367,55370985,7224,90.124277
2,ART0000042,42877050,9329,150.087551,55370985,7224,142.301489
3,ART0000043,21438525,9329,125.072959,27685492,7224,109.097809
4,ART0000115,32157787,9329,365.000000,166112956,7224,163.750000
...,...,...,...,...,...,...,...
408,ART0005109,96473362,9329,23.346952,55370985,7224,26.562945
409,ART0005110,96473362,9329,48.361544,27685492,7224,47.433830
410,ART0005111,75034837,9329,29.183690,27685492,7224,33.203681
411,ART0005112,75034837,9329,37.521888,27685492,7224,42.690447


In [71]:
sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x)/100000000000 from venditeS')

,sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x)/100000000000
0,507842.849547


In [72]:
x = sqldf('select NrArticolo,(Mix_y*VolumeTotale_y*PrezzoUnitario_y)/100000000000 as c from venditeS ')
x

,NrArticolo,c
0,ART0000018,85.380891
1,ART0000041,360.497103
2,ART0000042,569.205951
3,ART0000043,218.195611
4,ART0000115,1964.999990
...,...,...
408,ART0005109,106.251778
409,ART0005110,94.867657
410,ART0005111,66.407360
411,ART0005112,85.380891


In [73]:
BudgetStandardRic = sqldf('select sum(Mix_y*VolumeTotale_y*PrezzoUnitario_y)/100000000000 from venditeS ').iloc[0][0]
MixStandardRic = sqldf('select sum(Mix_y*VolumeTotale_x*PrezzoUnitario_y)/100000000000 from venditeS ').iloc[0][0]
MixEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_y)/100000000000 from venditeS ').iloc[0][0]
BudgetEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x)/100000000000 from venditeS ').iloc[0][0]


In [74]:
print(BudgetStandardRic)
print(MixStandardRic)
print(MixEffettivoRic)
print(BudgetEffettivoRic)

395335.8907839928
510532.7415730725
539112.4259562771
507842.84954721347


In [75]:
ScostamentoVolumeRic = MixStandardRic-BudgetStandardRic
ScostamentoMixRic = MixEffettivoRic-MixStandardRic
ScostamentoPrezzoRic = BudgetEffettivoRic-MixEffettivoRic

In [76]:
print(ScostamentoVolumeRic)
print(ScostamentoMixRic)
print(ScostamentoPrezzoRic)
print(ScostamentoVolumeRic+ScostamentoMixRic+ScostamentoPrezzoRic)

115196.85078907968
28579.684383204556
-31269.576409063593
112506.95876322064


# Vista globale degli scostamenti

In [77]:
data = {'Index': ['Ricavi', 'CostiMP', 'CostiLav', 'Margine'], 
        'Budget' : [BudgetStandardRic, BudgetStandardMP,BudgetStandardLav, (BudgetStandardRic - (BudgetStandardMP+BudgetStandardLav))],
        'ScostamentoVolume' : [ScostamentoVolumeRic,ScostamentoVolumeMP,ScostamentoVolumeLav, (ScostamentoVolumeRic - (ScostamentoVolumeMP+ScostamentoVolumeLav))], 
        'MixStandard' : [MixStandardRic, ImpiegoStandardMP,ImpiegoStandardLav, (MixStandardRic - (ImpiegoStandardMP+ImpiegoStandardLav))],
        'ScostamentoMix' : [ScostamentoMixRic,ScostamentoImpiegoMP,ScostamentoImpiegoLav, (ScostamentoMixRic - (ScostamentoImpiegoMP+ScostamentoImpiegoLav))],
        'MixEffettivo' : [MixEffettivoRic, ImpiegoEffettivoMP,ImpiegoEffettivoLav, (MixEffettivoRic - (ImpiegoEffettivoMP+ImpiegoEffettivoLav))],
        'ScostamentoPrezzo' : [ScostamentoPrezzoRic,ScostamentoPrezzoMP,ScostamentoPrezzoLav, (ScostamentoPrezzoRic - (ScostamentoPrezzoMP+ScostamentoPrezzoLav))], 
        'Consuntivo' : [BudgetEffettivoRic, BudgetEffettivoMP, BudgetEffettivoLav, (BudgetEffettivoRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
        'ScostamentoTotale' : [BudgetEffettivoRic-BudgetStandardRic,BudgetEffettivoMP-BudgetStandardMP,BudgetEffettivoLav-BudgetStandardLav, (BudgetEffettivoRic - BudgetStandardRic - (BudgetEffettivoMP-BudgetStandardMP+BudgetEffettivoLav-BudgetStandardLav))]}

In [78]:
pd.DataFrame(data)

,Index,Budget,ScostamentoVolume,MixStandard,ScostamentoMix,MixEffettivo,ScostamentoPrezzo,Consuntivo,ScostamentoTotale
0,Ricavi,395335.890784,115196.850789,510532.741573,28579.684383,539112.425956,-31269.576409,507842.849547,112506.958763
1,CostiMP,192906.470000,82983.236985,275889.706985,-2788.988729,273100.718257,16538.781743,289639.500000,96733.030000
2,CostiLav,194605.274842,148215.057329,342820.332170,-60981.672100,281838.660070,77511.885803,359350.545873,164745.271031
3,Margine,7824.145942,-116001.443525,-108177.297582,92350.345212,-15826.952370,-125320.243955,-141147.196326,-148971.342268


In [79]:
print(-119959.519152+96308.420839-125320.243955)


-148971.342268


# Repeat ma tenendo conto della valuta

In [80]:
venditeConsuntivoArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity,  sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as TotaleLocale, Valuta, TassoCambioMedio
        from venditeConsuntivo
        group by NrArticolo, Valuta, TassoCambioMedio''')
venditeBudgetArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity, sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as TotaleLocale, Valuta, TassoCambioMedio
        from venditeBudget
        group by NrArticolo, Valuta, TassoCambioMedio ''')

In [81]:
venditeBudgetArt

,NrArticolo,Quantity,TotaleLocale,Valuta,TassoCambioMedio
0,ART0000018,2,90.00,2,1.0541
1,ART0000041,4,380.00,2,1.0541
2,ART0000042,4,600.00,2,1.0541
3,ART0000043,2,230.00,2,1.0541
4,ART0000115,12,1965.00,1,1
...,...,...,...,...,...
410,ART0005111,2,70.00,2,1.0541
411,ART0005112,2,90.00,2,1.0541
412,ART0005136,87,8933.00,1,1
413,ART0005136,80,4175.21,2,1.0541


##### creazione colonne mix, volume, prezzo unitario (locale)

In [82]:
venditeConsuntivoS = sqldf('''select NrArticolo, (Quantity*100000000000/9329) as Mix, 9329 as VolumeTotale, TotaleLocale/Quantity as PrezzoUnitario, Valuta, TassoCambioMedio, (TotaleLocale/Quantity)/TassoCambioMedio as PrezzoUnitarioEuro
        from venditeConsuntivoArt''')
venditeBudgetS = sqldf('''select NrArticolo, (Quantity*100000000000/7224) as Mix, 7224 as VolumeTotale, TotaleLocale/Quantity as PrezzoUnitario, Valuta, TassoCambioMedio, (TotaleLocale/Quantity)/TassoCambioMedio as PrezzoUnitarioEuro
        from venditeBudgetArt''')


### Calcolo socostamenti di volume, mix, prezzo e valuta

In [83]:
venditeS1 = pd.merge(venditeConsuntivoS, venditeBudgetS, on = ['NrArticolo', 'Valuta'], how = 'outer')
venditeS1
#venditeS1.to_excel('venditeS1.xlsx')

,NrArticolo,Mix_x,VolumeTotale_x,PrezzoUnitario_x,Valuta,TassoCambioMedio_x,PrezzoUnitarioEuro_x,Mix_y,VolumeTotale_y,PrezzoUnitario_y,TassoCambioMedio_y,PrezzoUnitarioEuro_y
0,ART0000018,21438525,9329,55.000000,2,1.1993,45.860085,27685492,7224,45.000000,1.0541,42.690447
1,ART0000041,42877050,9329,120.000000,2,1.1993,100.058367,55370985,7224,95.000000,1.0541,90.124277
2,ART0000042,42877050,9329,180.000000,2,1.1993,150.087551,55370985,7224,150.000000,1.0541,142.301489
3,ART0000043,21438525,9329,150.000000,2,1.1993,125.072959,27685492,7224,115.000000,1.0541,109.097809
4,ART0000115,32157787,9329,365.000000,1,1,365.000000,166112956,7224,163.750000,1,163.750000
...,...,...,...,...,...,...,...,...,...,...,...,...
410,ART0005111,75034837,9329,35.000000,2,1.1993,29.183690,27685492,7224,35.000000,1.0541,33.203681
411,ART0005112,75034837,9329,45.000000,2,1.1993,37.521888,27685492,7224,45.000000,1.0541,42.690447
412,ART0005136,686032800,9329,39.109375,1,1,39.109375,1204318936,7224,102.678161,1,102.678161
413,ART0005136,439489763,9329,229.121951,2,1.1993,191.046403,1107419712,7224,52.190125,1.0541,49.511550


In [84]:
BudgetStandardRic = sqldf('select sum(Mix_y*VolumeTotale_y*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
MixStandardRic = sqldf('select sum(Mix_y*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
MixEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
BudgetEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
BudgetEffettivoValutaRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_x)/100000000000 from venditeS1 ').iloc[0][0]

In [85]:
print(BudgetStandardRic)
print(MixStandardRic)
print(MixEffettivoRic)
print(BudgetEffettivoRic)
print(BudgetEffettivoValutaRic)

395335.8907780576
510532.74156540784
513440.445133813
569389.7443994995
507842.8495423171


In [86]:
ScostamentoVolumeRic = MixStandardRic-BudgetStandardRic
ScostamentoMixRic = MixEffettivoRic-MixStandardRic
ScostamentoPrezzoRic = BudgetEffettivoRic-MixEffettivoRic
ScostamentoValutaRic = BudgetEffettivoValutaRic-BudgetEffettivoRic

In [87]:
print(ScostamentoVolumeRic)
print(ScostamentoMixRic)
print(ScostamentoPrezzoRic)
print(ScostamentoValutaRic)
print(ScostamentoVolumeRic+ScostamentoMixRic+ScostamentoPrezzoRic+ScostamentoValutaRic)

115196.85078735021
2907.7035684051807
55949.29926568648
-61546.8948571824
112506.95876425947


# Vista globale degli scostamenti v2

In [88]:
data = {'Index': ['Ricavi', 'CostiMP', 'CostiLav', 'Margine'], 
        'Budget' : [BudgetStandardRic, BudgetStandardMP,BudgetStandardLav, (BudgetStandardRic - (BudgetStandardMP+BudgetStandardLav))],
        'ScostamentoVolume' : [ScostamentoVolumeRic,ScostamentoVolumeMP,ScostamentoVolumeLav, (ScostamentoVolumeRic - (ScostamentoVolumeMP+ScostamentoVolumeLav))], 
        'MixStandard' : [MixStandardRic, ImpiegoStandardMP,ImpiegoStandardLav, (MixStandardRic - (ImpiegoStandardMP+ImpiegoStandardLav))],
        'ScostamentoMix' : [ScostamentoMixRic,ScostamentoImpiegoMP,ScostamentoImpiegoLav, (ScostamentoMixRic - (ScostamentoImpiegoMP+ScostamentoImpiegoLav))],
        'MixEffettivo' : [MixEffettivoRic, ImpiegoEffettivoMP,ImpiegoEffettivoLav, (MixEffettivoRic - (ImpiegoEffettivoMP+ImpiegoEffettivoLav))],
        'ScostamentoPrezzo' : [ScostamentoPrezzoRic,ScostamentoPrezzoMP,ScostamentoPrezzoLav, (ScostamentoPrezzoRic - (ScostamentoPrezzoMP+ScostamentoPrezzoLav))], 
        'MixValuta' : [BudgetEffettivoRic, BudgetEffettivoMP,BudgetEffettivoLav, (BudgetEffettivoRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
        'ScostamentoValuta' : [ScostamentoValutaRic, 0, 0, (ScostamentoValutaRic - (0+0))],
        'Consuntivo' : [BudgetEffettivoValutaRic, BudgetEffettivoMP, BudgetEffettivoLav, (BudgetEffettivoValutaRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
        'ScostamentoTotale' : [BudgetEffettivoValutaRic-BudgetStandardRic,BudgetEffettivoMP-BudgetStandardMP,BudgetEffettivoLav-BudgetStandardLav, (BudgetEffettivoValutaRic - BudgetStandardRic - (BudgetEffettivoMP-BudgetStandardMP+BudgetEffettivoLav-BudgetStandardLav))]}

In [89]:
pd.DataFrame(data)

,Index,Budget,ScostamentoVolume,MixStandard,ScostamentoMix,MixEffettivo,ScostamentoPrezzo,MixValuta,ScostamentoValuta,Consuntivo,ScostamentoTotale
0,Ricavi,395335.890778,115196.850787,510532.741565,2907.703568,513440.445134,55949.299266,569389.744399,-61546.894857,507842.849542,112506.958764
1,CostiMP,192906.470000,82983.236985,275889.706985,-2788.988729,273100.718257,16538.781743,289639.500000,0.000000,289639.500000,96733.030000
2,CostiLav,194605.274842,148215.057329,342820.332170,-60981.672100,281838.660070,77511.885803,359350.545873,0.000000,359350.545873,164745.271031
3,Margine,7824.145936,-116001.443527,-108177.297590,66678.364397,-41498.933193,-38101.368281,-79600.301473,-61546.894857,-141147.196331,-148971.342267


come vedremo, i volumi di vendita non coincidono ai volumi di produzione, come negli esercizi fatti a lezione.

Invece, a budget, si vendenono circa 412 prodotti diversi, mentre si lavorano circa 503, dei quali soltanto 73 passano per una fase di montaggio nell'anno in quesitone.

Questo ci porta alla conclusione che ci sono delle rimanenze, e non tutto cio che e prodotto nell'anno e venduto nello stesso anno.

In [90]:
impiegoBudgetNew

,NrArticolo,Descrizione,NrOrdineProduzione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,Volume,QuantitydiOutput
0,ART0000018,Controllo qualità,ODP17-05342,CQ,0.00,0.000000,12.750000,6.0,2.0
1,ART0000018,Fresatura,ODP17-05342,A30,0.75,0.125000,51.000000,6.0,2.0
2,ART0000018,Tornitura,ODP17-05342,A20,1.00,0.166667,68.000000,6.0,2.0
3,ART0000041,Controllo qualità,ODP17-05342,CQ,0.00,0.000000,12.750000,12.0,4.0
4,ART0000041,Fresatura,ODP17-05342,A30,1.25,0.104167,51.000000,12.0,4.0
...,...,...,...,...,...,...,...,...,...
1222,ART0005136,Foratura/Maschiatura,ODP17-00667,A32,3.00,0.004559,42.500000,658.0,0.0
1223,ART0005136,Fresatura,ODP16-16477,A30,21.33,0.032416,47.600000,658.0,22.0
1224,ART0005136,Montaggio,ODP17-00667,A40,8.25,0.012538,42.500000,658.0,0.0
1225,ART0005136,Taglio spezzoni,ODP17-14994,A20,4.50,0.006839,68.000000,658.0,29.0


In [91]:
sqldf('select NrArticolo, QuantitydiOutput from impiegoBudgetNew group by NrArticolo,QuantitydiOutput order by NrArticolo')

,NrArticolo,QuantitydiOutput
0,ART0000018,2.0
1,ART0000041,4.0
2,ART0000042,4.0
3,ART0000043,2.0
4,ART0000115,6.0
...,...,...
640,ART0005136,22.0
641,ART0005136,29.0
642,ART0005136,34.0
643,ART0005136,272.0


In [92]:
sqldf('select NrArticolo,Quantity from venditeBudget group by NrArticolo,Quantity order by NrArticolo')

,NrArticolo,Quantity
0,ART0000018,2
1,ART0000041,4
2,ART0000042,4
3,ART0000043,2
4,ART0000115,1
...,...,...
585,ART0005136,30
586,ART0005136,32
587,ART0005136,42
588,ART0005136,50


In [93]:
sqldf('select NrArticolo,sum(Quantity) as quantita from venditeBudget group by NrArticolo order by NrArticolo')

,NrArticolo,quantita
0,ART0000018,2
1,ART0000041,4
2,ART0000042,4
3,ART0000043,2
4,ART0000115,12
...,...,...
408,ART0005109,4
409,ART0005110,2
410,ART0005111,2
411,ART0005112,2


In [94]:
sqldf("SELECT DISTINCT NrArticolo FROM dfImpiegoRisorse WHERE (budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget')")

,NrArticolo
0,ART0000018
1,ART0000041
2,ART0000042
3,ART0000043
4,ART0000115
...,...
499,ART0005109
500,ART0005110
501,ART0005111
502,ART0005112


In [95]:
sqldf("SELECT DISTINCT NrArticolo,QuantitydiOutput FROM dfImpiegoRisorse WHERE (budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget') and Descrizione = 'Montaggio'")

,NrArticolo,QuantitydiOutput
0,ART0000748,50.0
1,ART0000748,10.0
2,ART0000748,30.0
3,ART0000752,1.0
4,ART0000752,6.0
...,...,...
69,ART0004099,1.0
70,ART0004104,4.0
71,ART0004146,31.0
72,ART0004146,0.0


In [96]:
pd.merge(
    sqldf('select NrArticolo,sum(Quantity) as quantita from venditeBudget group by NrArticolo order by NrArticolo'),
    sqldf("SELECT DISTINCT NrArticolo,QuantitydiOutput FROM dfImpiegoRisorse WHERE (budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget') and Descrizione = 'Montaggio'"), 
    on='NrArticolo', how='outer')

,NrArticolo,quantita,QuantitydiOutput
0,ART0000018,2,NaN
1,ART0000041,4,NaN
2,ART0000042,4,NaN
3,ART0000043,2,NaN
4,ART0000115,12,NaN
...,...,...,...
439,ART0005109,4,NaN
440,ART0005110,2,NaN
441,ART0005111,2,NaN
442,ART0005112,2,NaN


In [97]:
venditeConsuntivo.to_excel('venditeConsuntivo.xlsx')

# By Criterion

In [98]:
def by_client(dfVendite, client):
        dfVendite.rename(columns = {'budget/cons':'budget'}, inplace = True)
        venditeConsuntivo = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'CONSUNTIVO' or budget = 'Consuntivo'")
        venditeBudget = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'BUDGET' or budget = 'Budget'")

        venditeConsuntivo = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
        FROM venditeConsuntivo as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
        where k.Anno = 'Consuntivo' or k.Anno = 'CONSUNTIVO'
        ''')

        venditeBudget = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
        FROM venditeBudget as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
        where k.Anno = 'Budget' or k.Anno = 'BUDGET'
        ''')

        venditeConsuntivo = sqldf('select * from venditeConsuntivo where NrOrigine = "'+client+'"')

        venditeBudget = sqldf('select * from venditeBudget where NrOrigine = "'+client+'"')

        venditeConsuntivoArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity,  sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as TotaleLocale, Valuta, TassoCambioMedio
                from venditeConsuntivo
                group by NrArticolo, Valuta, TassoCambioMedio''')
        venditeBudgetArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity, sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as TotaleLocale, Valuta, TassoCambioMedio
                from venditeBudget
                group by NrArticolo, Valuta, TassoCambioMedio ''')

        venditeConsuntivoS = sqldf('''select NrArticolo, (Quantity*100000000000/9329) as Mix, 9329 as VolumeTotale, TotaleLocale/Quantity as PrezzoUnitario, Valuta, TassoCambioMedio, (TotaleLocale/Quantity)/TassoCambioMedio as PrezzoUnitarioEuro
                from venditeConsuntivoArt''')
        venditeBudgetS = sqldf('''select NrArticolo, (Quantity*100000000000/7224) as Mix, 7224 as VolumeTotale, TotaleLocale/Quantity as PrezzoUnitario, Valuta, TassoCambioMedio, (TotaleLocale/Quantity)/TassoCambioMedio as PrezzoUnitarioEuro
                from venditeBudgetArt''')

        venditeS1 = pd.merge(venditeConsuntivoS, venditeBudgetS, on = ['NrArticolo', 'Valuta'], how = 'outer')
        venditeS1.fillna(0, inplace = True)
        venditeS1


        BudgetStandardRic = sqldf('select sum(Mix_y*VolumeTotale_y*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
        MixStandardRic = sqldf('select sum(Mix_y*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
        MixEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
        BudgetEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
        BudgetEffettivoValutaRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_x)/100000000000 from venditeS1 ').iloc[0][0]

        if(BudgetStandardRic is None):
           BudgetStandardRic = 0

        if(BudgetEffettivoValutaRic is None):
          BudgetEffettivoValutaRic = 0

        ScostamentoVolumeRic = MixStandardRic-BudgetStandardRic
        ScostamentoMixRic = MixEffettivoRic-MixStandardRic
        ScostamentoPrezzoRic = BudgetEffettivoRic-MixEffettivoRic
        ScostamentoValutaRic = BudgetEffettivoValutaRic-BudgetEffettivoRic

        dicResult = {
        'BudgetStandardRic':BudgetStandardRic,
        'MixStandardRic':MixStandardRic,
        'MixEffettivoRic':MixEffettivoRic,
        'BudgetEffettivoRic':BudgetEffettivoRic,
        'BudgetEffettivoValutaRic':BudgetEffettivoValutaRic,
        'ScostamentoVolumeRic':ScostamentoVolumeRic,
        'ScostamentoMixRic':ScostamentoMixRic,
        'ScostamentoPrezzoRic':ScostamentoPrezzoRic,
        'ScostamentoValutaRic':ScostamentoValutaRic,
        'ScostamentoTotaleRic' : ScostamentoVolumeRic+ScostamentoMixRic+ScostamentoPrezzoRic+ScostamentoValutaRic
        }

        return dicResult




In [99]:
e = by_client(dfVendite, 'C00471')
e


{'BudgetStandardRic': 14292.79945018925,
 'MixStandardRic': 18457.57559119816,
 'MixEffettivoRic': 37328.980168508795,
 'BudgetEffettivoRic': 39465.894976324234,
 'BudgetEffettivoValutaRic': 34687.734423866736,
 'ScostamentoVolumeRic': 4164.7761410089115,
 'ScostamentoMixRic': 18871.404577310634,
 'ScostamentoPrezzoRic': 2136.9148078154394,
 'ScostamentoValutaRic': -4778.160552457499,
 'ScostamentoTotaleRic': 20394.934973677486}

In [100]:
def by_client(dfVendite,dfConsumi,impiegoConsuntivo1, impiegoBudget1, client):

       dfVendite.rename(columns = {'budget/cons':'budget'}, inplace = True)
       venditeConsuntivo = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'CONSUNTIVO' or budget = 'Consuntivo'")
       venditeBudget = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'BUDGET' or budget = 'Budget'")

       venditeConsuntivo = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
       FROM venditeConsuntivo as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
       where k.Anno = 'Consuntivo' or k.Anno = 'CONSUNTIVO'
       ''')

       venditeBudget = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
       FROM venditeBudget as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
       where k.Anno = 'Budget' or k.Anno = 'BUDGET'
       ''')

       venditeConsuntivo = sqldf('select * from venditeConsuntivo where NrOrigine = "'+client+'"')

       venditeBudget = sqldf('select * from venditeBudget where NrOrigine = "'+client+'"')

       venditeConsuntivoArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity,  sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as TotaleLocale, Valuta, TassoCambioMedio
       from venditeConsuntivo
       group by NrArticolo, Valuta, TassoCambioMedio''')
       venditeBudgetArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity, sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as TotaleLocale, Valuta, TassoCambioMedio
       from venditeBudget
       group by NrArticolo, Valuta, TassoCambioMedio ''')

       venditeConsuntivoS = sqldf('''select NrArticolo, (Quantity*100000000000/9329) as Mix, 9329 as VolumeTotale, TotaleLocale/Quantity as PrezzoUnitario, Valuta, TassoCambioMedio, (TotaleLocale/Quantity)/TassoCambioMedio as PrezzoUnitarioEuro
       from venditeConsuntivoArt''')
       venditeBudgetS = sqldf('''select NrArticolo, (Quantity*100000000000/7224) as Mix, 7224 as VolumeTotale, TotaleLocale/Quantity as PrezzoUnitario, Valuta, TassoCambioMedio, (TotaleLocale/Quantity)/TassoCambioMedio as PrezzoUnitarioEuro
       from venditeBudgetArt''')

       venditeS1 = pd.merge(venditeConsuntivoS, venditeBudgetS, on = ['NrArticolo', 'Valuta'], how = 'outer')
       venditeS1.fillna(0, inplace = True)
       venditeS1


       BudgetStandardRic = sqldf('select sum(Mix_y*VolumeTotale_y*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
       MixStandardRic = sqldf('select sum(Mix_y*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
       MixEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
       BudgetEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
       BudgetEffettivoValutaRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_x)/100000000000 from venditeS1 ').iloc[0][0]

       if(BudgetStandardRic is None):
         BudgetStandardRic = 0

       if(MixStandardRic is None):
               MixStandardRic = 0
       
       if(MixEffettivoRic is None):
               MixEffettivoRic = 0
       
       if(BudgetEffettivoRic is None):
               BudgetEffettivoRic = 0

       if(BudgetEffettivoValutaRic is None):
         BudgetEffettivoValutaRic = 0

       ScostamentoVolumeRic = MixStandardRic-BudgetStandardRic
       ScostamentoMixRic = MixEffettivoRic-MixStandardRic
       ScostamentoPrezzoRic = BudgetEffettivoRic-MixEffettivoRic
       ScostamentoValutaRic = BudgetEffettivoValutaRic-BudgetEffettivoRic

       

       # materia prima

       articoli = sqldf('select distinct NrArticolo from venditeS1')

       dfConsumi.rename(columns = {'Budget/cons':'budget'}, inplace = True)
       consumiConsuntivo = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'CONSUNTIVO' and NrArticolo in (select * from articoli)")
       consumiBudget = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'BUDGET' and NrArticolo in (select * from articoli)") 

       consumiConsuntivoUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
       from consumiConsuntivo
       order by CodiceMP''')
       consumiBudgetUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
       from consumiBudget
       order by CodiceMP''')

       consumiConsuntivoUnitByMP = sqldf('''select CodiceMP, NrArticolo,938999.16 as Volume, sum(QuantityMPImpiegata) as QuantityMPImpiegata,sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario , sum(ImportoCostoTOTALE) as ImportoCostoTOTALE
       from consumiConsuntivoUnit      
       group by CodiceMP, NrArticolo''')
       consumiBudgetUnitByMP = sqldf('''select CodiceMP, NrArticolo,550834.0 as Volume, sum(QuantityMPImpiegata) as QuantityMPImpiegata,sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario , sum(ImportoCostoTOTALE) as ImportoCostoTOTALE
       from consumiBudgetUnit      
       group by CodiceMP, NrArticolo''')

       conVol = sqldf('select CodiceMP, sum(QuantityMPImpiegata) as VolumeByMP from consumiConsuntivoUnitByMP group by CodiceMP')
       buVol = sqldf('select CodiceMP, sum(QuantityMPImpiegata) as VolumeByMP from consumiBudgetUnitByMP group by CodiceMP')

       consumiConsuntivoUnitByMP = sqldf('''select CodiceMP, NrArticolo, VolumeByMP, (QuantityMPImpiegata/VolumeByMP) as Impiego, CostoUnitario, ImportoCostoTOTALE
       from consumiConsuntivoUnitByMP natural join conVol ''')
       consumiBudgetUnitByMP = sqldf('''select CodiceMP, NrArticolo, VolumeByMP, (QuantityMPImpiegata/VolumeByMP) as Impiego, CostoUnitario, ImportoCostoTOTALE
       from consumiBudgetUnitByMP natural join buVol''')

       ConsumiUnit = pd.merge(consumiConsuntivoUnitByMP,consumiBudgetUnitByMP, on = ['CodiceMP','NrArticolo'], how = 'outer')

       BudgetEffettivoMP = sqldf('select sum(VolumeByMP_x*Impiego_x*CostoUnitario_x) from ConsumiUnit').iloc[0][0]
       ImpiegoEffettivoMP = sqldf('select sum(VolumeByMP_x*Impiego_x*CostoUnitario_y) from ConsumiUnit').iloc[0][0]
       ImpiegoStandardMP = sqldf('select sum(VolumeByMP_x*Impiego_y*CostoUnitario_y) from ConsumiUnit').iloc[0][0]
       BudgetStandardMP = sqldf('select sum(VolumeByMP_y*Impiego_y*CostoUnitario_y) from ConsumiUnit').iloc[0][0]
       ScostamentoVolumeMP =  ImpiegoStandardMP - BudgetStandardMP
       ScostamentoImpiegoMP = ImpiegoEffettivoMP-ImpiegoStandardMP
       ScostamentoPrezzoMP = BudgetEffettivoMP-ImpiegoEffettivoMP 

       # lavorazioni

       impiegoConsuntivoNew = sqldf(''' select NrArticolo, budgetConsuntivo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa, TempoRisorsa, TempoRisorsa/QuantitydiOutput as ImpiegoUnitario, CostoOrario ,QuantitydiOutput
       from impiegoConsuntivo1
       where NrArticolo in (select * from articoli)''')
       impiegoConsuntivoNew.drop(['NrOrdineProduzione'], axis=1, inplace=True)

       impiegoConsuntivoNew = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, sum(TempoRisorsa) as TempoRisorsa, avg(CostoOrario) as CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
       from impiegoConsuntivoNew
       group by NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione
       ''')

       vol = sqldf('select NrArticolo, sum(QuantitydiOutput) as Volume from impiegoConsuntivoNew group by NrArticolo')

       impiegoConsuntivoNew = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, TempoRisorsa, (TempoRisorsa/Volume) as ImpiegoUnitario, CostoOrario, Volume ,QuantitydiOutput
       from impiegoConsuntivoNew natural join vol
       ''')

       impiegoBudgetNew = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, sum(TempoRisorsa) as TempoRisorsa, avg(CostoOrario) as CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
       from impiegoBudget1
       where NrArticolo in (select * from articoli)
       group by NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione
       ''')
       vol = sqldf('select NrArticolo, sum(QuantitydiOutput) as Volume from impiegoBudgetNew group by NrArticolo')

       impiegoBudgetNew = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, TempoRisorsa, (TempoRisorsa/Volume) as ImpiegoUnitario, CostoOrario, Volume ,QuantitydiOutput
       from impiegoBudgetNew natural join vol
       ''')

       impiegoConsuntivoNew.drop(['budgetConsuntivo'], axis=1, inplace=True)
       impiegoBudgetNew.drop(['budgetConsuntivo'], axis=1, inplace=True)
       x = pd.merge(impiegoConsuntivoNew, impiegoBudgetNew, on=['NrArticolo', 'Descrizione', 'NrAreaProduzione'], how='outer')

       #Budget Standard
       BudgetStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_y	) from x').iloc[0][0]
       # Impiego Standard (volume a consuntivo)
       ImpiegoStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_x	) from x').iloc[0][0]
       # Impiego Effettivo (volume e impiego a consuntivo)
       ImpiegoEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_y*QuantitydiOutput_x	) from x').iloc[0][0]
       # Budget Effettivo (tutto a consuntivo)
       BudgetEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*QuantitydiOutput_x	) from x').iloc[0][0]

       ScostamentoVolumeLav = ImpiegoStandardLav-b
       ScostamentoImpiegoLav = ImpiegoEffettivoLav-ImpiegoStandardLav
       ScostamentoPrezzoLav = BudgetEffettivoLav-ImpiegoEffettivoLav

       data = {client: ['Ricavi', 'CostiMP', 'CostiLav', 'Margine'], 
       'Budget' : [BudgetStandardRic, BudgetStandardMP,BudgetStandardLav, (BudgetStandardRic - (BudgetStandardMP+BudgetStandardLav))],
       'ScostamentoVolume' : [ScostamentoVolumeRic,ScostamentoVolumeMP,ScostamentoVolumeLav, (ScostamentoVolumeRic - (ScostamentoVolumeMP+ScostamentoVolumeLav))], 
       'MixStandard' : [MixStandardRic, ImpiegoStandardMP,ImpiegoStandardLav, (MixStandardRic - (ImpiegoStandardMP+ImpiegoStandardLav))],
       'ScostamentoMix' : [ScostamentoMixRic,ScostamentoImpiegoMP,ScostamentoImpiegoLav, (ScostamentoMixRic - (ScostamentoImpiegoMP+ScostamentoImpiegoLav))],
       'MixEffettivo' : [MixEffettivoRic, ImpiegoEffettivoMP,ImpiegoEffettivoLav, (MixEffettivoRic - (ImpiegoEffettivoMP+ImpiegoEffettivoLav))],
       'ScostamentoPrezzo' : [ScostamentoPrezzoRic,ScostamentoPrezzoMP,ScostamentoPrezzoLav, (ScostamentoPrezzoRic - (ScostamentoPrezzoMP+ScostamentoPrezzoLav))], 
       'MixValuta' : [BudgetEffettivoRic, BudgetEffettivoMP,BudgetEffettivoLav, (BudgetEffettivoRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
       'ScostamentoValuta' : [ScostamentoValutaRic, 0, 0, (ScostamentoValutaRic - (0+0))],
       'Consuntivo' : [BudgetEffettivoValutaRic, BudgetEffettivoMP, BudgetEffettivoLav, (BudgetEffettivoValutaRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
       'ScostamentoTotale' : [BudgetEffettivoValutaRic-BudgetStandardRic,BudgetEffettivoMP-BudgetStandardMP,BudgetEffettivoLav-BudgetStandardLav, (BudgetEffettivoValutaRic - BudgetStandardRic - (BudgetEffettivoMP-BudgetStandardMP+BudgetEffettivoLav-BudgetStandardLav))]}

       data = pd.DataFrame(data)

       return data




In [101]:
clients = sqldf('select distinct Nr from dfClienti')
scos = []
count = 0

for client in clients.values:
   theclient = client[0]
   if (sqldf('select * from dfVendite where NrOrigine = "'+theclient+'"').empty):
      continue
   print(theclient)
   scos.append(by_client(dfVendite,dfConsumi,impiegoConsuntivo1, impiegoBudget1, theclient))
   count += 1

C00001
C00003
C00004
C00015
C00017
C00018
C00020
C00026
C00084
C00094
C00117
C00127
C00140
C00158
C00236
C00297
C00344
C00353
C00454
C00455


In [ ]:
def Func(x):
  return x.iloc[3,10]

scos.sort(key=Func)

In [ ]:
scos[0]

,C00140,Budget,ScostamentoVolume,MixStandard,ScostamentoMix,MixEffettivo,ScostamentoPrezzo,MixValuta,ScostamentoValuta,Consuntivo,ScostamentoTotale
0,Ricavi,339345.250517,98881.748663,438226.999179,-31816.279916,406410.719263,50383.994864,456794.714128,-55304.421322,401490.292806,62145.042289
1,CostiMP,148088.440000,74185.889444,222274.329444,-531.712890,221742.616554,2248.293446,223990.910000,0.000000,223990.910000,75902.470000
2,CostiLav,76429.574667,-65657.993183,128947.281659,-16680.978656,112266.303003,27124.534370,139390.837373,0.000000,139390.837373,62961.262707
3,Margine,114827.235850,90353.852401,87005.388076,-14603.588371,72401.799705,21011.167049,93412.966754,-55304.421322,38108.545432,-76718.690418


# Scratchpad

In [ ]:
y = sqldf('select NrArticolo,count(descrizione) from impiegoBudget1 group by NrArticolo having Descrizione = "Controllo qualità" or Descrizione = "Montaggio"')
y

,NrArticolo,count(descrizione)
0,ART0000018,3
1,ART0000041,3
2,ART0000042,3
3,ART0000121,1
4,ART0000125,7
...,...,...
432,ART0005109,2
433,ART0005110,1
434,ART0005111,2
435,ART0005112,2


In [ ]:
x = sqldf('select NrArticolo,count(descrizione) from impiegoBudget1 group by NrArticolo')
x

,NrArticolo,count(descrizione)
0,ART0000018,3
1,ART0000041,3
2,ART0000042,3
3,ART0000043,4
4,ART0000115,2
...,...,...
499,ART0005109,2
500,ART0005110,1
501,ART0005111,2
502,ART0005112,2


In [ ]:
sqldf ('select NrArticolo from x where NrArticolo not in (select NrArticolo from y)')

,NrArticolo
0,ART0000043
1,ART0000115
2,ART0000116
3,ART0000119
4,ART0000157
...,...
62,ART0004363
63,ART0004836
64,ART0004886
65,ART0004941
